# 🏇 JRA 全レース取得 (2020-2026)
以下の設定変数を変更して実行してください。指定した期間のデータを取得し、`SAVE_DIR` に保存します。

In [ ]:
# Google Driveをマウントする場合のみ実行してください
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 設定 (ここを変更してください)
YEAR = 2024          # 対象年度 (例: 2024)
START_MONTH = 1      # 開始月 (1-12)
END_MONTH = 12       # 終了月 (1-12)
SAVE_DIR = '/content/drive/MyDrive/dai-keiba/data/raw' # 保存先フォルダ

# 実行ブロック
import os
from datetime import date
import calendar

if YEAR:
    # Saveディレクトリの作成
    os.makedirs(SAVE_DIR, exist_ok=True)
    
    s_date = date(int(YEAR), int(START_MONTH), 1)
    last_day = calendar.monthrange(int(YEAR), int(END_MONTH))[1]
    e_date = date(int(YEAR), int(END_MONTH), last_day)
    
    # 未来の日付は検索しないように制限
    today = date.today()
    if e_date > today:
        e_date = today
    
    save_path = os.path.join(SAVE_DIR, 'database.csv')
    print(f'{YEAR}年のデータを {s_date} から {e_date} まで取得します...')
    print(f'保存先: {save_path}')
    
    # 既存データの読み込み (重複取得防止)
    existing_race_ids = None
    if os.path.exists(save_path):
        try:
            df_exist = pd.read_csv(save_path, usecols=['race_id'])
            existing_race_ids = set(df_exist['race_id'].astype(str))
            print(f'既存データ: {len(existing_race_ids)} 件のレースIDを読み込みました。')
        except Exception as e:
            print(f'既存データの読み込みに失敗しました: {e}')
    
    # 安全な追記関数 (カラムずれ防止)
    def safe_append_csv(df_chunk, path):
        import pandas as pd
        import os
        if not os.path.exists(path):
            df_chunk.to_csv(path, index=False)
        else:
            try:
                # 既存ヘッダー読み込み
                existing_cols = pd.read_csv(path, nrows=0).columns.tolist()
                # カラム合わせ (過不足対応)
                df_aligned = df_chunk.reindex(columns=existing_cols)
                # 追記
                df_aligned.to_csv(path, mode='a', header=False, index=False)
            except Exception as e:
                print(f"Save Error: {e}")
                # 万が一の場合はバックアップして新規作成するなどの分岐も可だが、ここではエラー表示のみ

    scrape_jra_year_rich(str(YEAR), start_date=s_date, end_date=e_date, save_callback=lambda df: safe_append_csv(df, save_path), existing_race_ids=existing_race_ids)
    print('完了しました。')
else:
    print('年度が設定されていません。')
